In [1]:
# Import packages
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import make_scorer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import SMOTE
from collections import Counter
from parse import preprocess

In [2]:
# Function to show classification report for Cross Validation
def classification_report_with_accuracy_score(y_true, y_pred):
    print(classification_report(y_true, y_pred)) # print classification report
    return accuracy_score(y_true, y_pred) # return accuracy score

In [3]:
# Pre-parse the dataset
data = preprocess("rawfile_blood.csv")

robust          368
prefrail_mci    268
prefrail        250
mci             142
frail_mci        86
frail             9
Name: condition, dtype: int64

####################################################################
Number of Rows of Dataframe:
1123
Number of Columns of Dataframe:
59

####################################################################
Threshold for number of NULLs in a column: 0.1095
Number of Columns before Parsing for Too Many NULLs in a column:
59
Number of Columns after Parsing for Too Many NULLs in a column:
51

Columns Removed:
B1_b5
B4_a1
B4_a3
B4_a4
B4_a6
B4_b1
B4_b3
B5_a1

####################################################################
Number of Columns after dropping A1_2, B1_b4, B2_c3, B4_b2 for inconsistent data types:
47

####################################################################
Number of Rows before Parsing NULLs in data:
1123
Number of Rows after Parsing NULLs in data:
1015


In [4]:
data.head()

,mtag,condition,A1_1,A2_1,A3_1,B1_a,B1_a1,B1_a2,B1_a3,B1_a4,...,B2_d6,B2_d7,B2_d8,B2_d9,B3,B4_a2,B4_a5,B5_a2,B5_a3,B6
0,ME02646,frail,196,24,46.5,121,3.93,0.37,95,31,...,7,12,13,6,0.2,6.0,1.011,1.14,4.1,5.9
1,ME03109,frail,200,23,55.6,142,4.82,0.42,87,30,...,7,20,17,26,3.1,5.0,1.011,3.25,4.6,8.5
2,ME06997,frail,441,20,76.8,105,4.54,0.41,90,30,...,5,16,19,15,1.4,7.0,1.023,2.14,4.0,6.4
3,ME07149,frail,265,16,47.2,122,4.53,0.39,86,27,...,8,24,19,21,2.1,5.5,1.012,1.06,4.7,6.1
4,ME07700,frail,425,14,31.3,124,4.44,0.38,85,28,...,6,20,23,23,6.0,5.5,1.013,1.95,3.8,5.8


In [5]:
data.columns

Index(['mtag', 'condition', 'A1_1', 'A2_1', 'A3_1', 'B1_a', 'B1_a1', 'B1_a2',
       'B1_a3', 'B1_a4', 'B1_a5', 'B1_a6', 'B1_b', 'B1_b1', 'B1_b2', 'B1_b3',
       'B1_c', 'B1_d', 'B2_a1', 'B2_a2', 'B2_a3', 'B2_a4', 'B2_a5', 'B2_b1',
       'B2_b2', 'B2_b3', 'B2_c1', 'B2_c2', 'B2_c4', 'B2_c5', 'B2_c6', 'B2_c7',
       'B2_d1', 'B2_d2', 'B2_d3', 'B2_d4', 'B2_d5', 'B2_d6', 'B2_d7', 'B2_d8',
       'B2_d9', 'B3', 'B4_a2', 'B4_a5', 'B5_a2', 'B5_a3', 'B6'],
      dtype='object')

In [6]:
c = data['condition'].value_counts()
condition = c.index
c

robust          343
prefrail_mci    233
prefrail        223
mci             133
frail_mci        76
frail             7
Name: condition, dtype: int64

In [7]:
for i in range(len(condition)):
    data['condition'].replace(condition[i], i, inplace = True)

data.head()

,mtag,condition,A1_1,A2_1,A3_1,B1_a,B1_a1,B1_a2,B1_a3,B1_a4,...,B2_d6,B2_d7,B2_d8,B2_d9,B3,B4_a2,B4_a5,B5_a2,B5_a3,B6
0,ME02646,5,196,24,46.5,121,3.93,0.37,95,31,...,7,12,13,6,0.2,6.0,1.011,1.14,4.1,5.9
1,ME03109,5,200,23,55.6,142,4.82,0.42,87,30,...,7,20,17,26,3.1,5.0,1.011,3.25,4.6,8.5
2,ME06997,5,441,20,76.8,105,4.54,0.41,90,30,...,5,16,19,15,1.4,7.0,1.023,2.14,4.0,6.4
3,ME07149,5,265,16,47.2,122,4.53,0.39,86,27,...,8,24,19,21,2.1,5.5,1.012,1.06,4.7,6.1
4,ME07700,5,425,14,31.3,124,4.44,0.38,85,28,...,6,20,23,23,6.0,5.5,1.013,1.95,3.8,5.8


In [8]:
data.tail

<bound method NDFrame.tail of          mtag  condition  A1_1  A2_1  A3_1  B1_a  B1_a1  B1_a2  B1_a3  B1_a4  \
0     ME02646          5   196    24  46.5   121   3.93   0.37     95     31   
1     ME03109          5   200    23  55.6   142   4.82   0.42     87     30   
2     ME06997          5   441    20  76.8   105   4.54   0.41     90     30   
3     ME07149          5   265    16  47.2   122   4.53   0.39     86     27   
4     ME07700          5   425    14  31.3   124   4.44   0.38     85     28   
...       ...        ...   ...   ...   ...   ...    ...    ...    ...    ...   
1010  MV00454          0   220    19  67.5   138   4.66   0.42     91     30   
1011  MV00456          0   334    18  51.0   139   4.63   0.42     91     30   
1012  MV00460          0   418    17  61.0   122   4.18   0.38     90     29   
1013  MV00502          0   393    18  43.1   136   4.57   0.43     94     30   
1014  MV00510          0   371    24  55.9   127   4.41   0.40     90     29   

      ...

In [10]:
y = data['condition']

features = ['A2_1', 'B2_d2', 'B5_a3', 'B6']

X = data[features]

X = StandardScaler().fit_transform(X)
X = MinMaxScaler().fit_transform(X)

In [11]:
# Summarise the new class distribution
counter = Counter(y)
print(counter)

Counter({0: 343, 1: 233, 2: 223, 3: 133, 4: 76, 5: 7})


In [12]:
# Test 1: Splitting the dataset into separate train and test sets

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.4, random_state = 1)

# Logistic Regression

log_model = LogisticRegression()
log_model.fit(X_train, y_train)
print("Logistic Regression:", log_model.score(X_test, y_test).round(3))

# Linear Discriminant Analysis

lda_model = LinearDiscriminantAnalysis()
lda_model.fit(X_train, y_train)
print("Linear Discriminant Analysis:", lda_model.score(X_test, y_test).round(3))

# K-Nearest Neigbors

knn_model = KNeighborsClassifier()
knn_model.fit(X_train, y_train)
print("K-Nearest Neigbors:", knn_model.score(X_test, y_test).round(3))

# Classification and Regression Trees

cart_model = DecisionTreeClassifier()
cart_model.fit(X_train, y_train)
print("Classification and Regression Trees:", cart_model.score(X_test, y_test).round(3))

# Gaussian Naive Bayes

gnb_model = GaussianNB()
gnb_model.fit(X_train, y_train)
print("Gaussian Naive Bayes:", gnb_model.score(X_test, y_test).round(3))

# Support Vector Machines

svm_model = SVC(kernel='linear', gamma = 'auto')
svm_model.fit(X_train, y_train)
print("Support Vector Machines:", svm_model.score(X_test, y_test).round(3))

# Random Forest Classifier

rfc_model = RandomForestClassifier()
rfc_model.fit(X_train, y_train)
print("Random Forest Classifier:", rfc_model.score(X_test, y_test).round(3))

Logistic Regression: 0.374
Linear Discriminant Analysis: 0.372
K-Nearest Neigbors: 0.303
Classification and Regression Trees: 0.232
Gaussian Naive Bayes: 0.347
Support Vector Machines: 0.362
Random Forest Classifier: 0.251


In [14]:
# Test 2: 5-fold Cross Validation

In [15]:
# Logistic Regression

log_model = LogisticRegression()
log_model.fit(X, y)
scores = cross_val_score(log_model, X, y, cv=5)
print("Logistic Regression: %0.2f accuracy with a standard deviation of %0.2f" % (scores.mean(), scores.std()))

# Linear Discriminant Analysis

lda_model = LinearDiscriminantAnalysis()
lda_model.fit(X, y)
scores = cross_val_score(lda_model, X, y, cv=5)
print("Linear Discriminant Analysis: %0.2f accuracy with a standard deviation of %0.2f" % (scores.mean(), scores.std()))

# K-Nearest Neigbors

knn_model = KNeighborsClassifier()
knn_model.fit(X, y)
scores = cross_val_score(knn_model, X, y, cv=5)
print("K-Nearest Neighbors: %0.2f accuracy with a standard deviation of %0.2f" % (scores.mean(), scores.std()))

# Classification and Regression Trees

cart_model = DecisionTreeClassifier()
cart_model.fit(X, y)
scores = cross_val_score(cart_model, X, y, cv=5)
print("Classification and Regression Trees: %0.2f accuracy with a standard deviation of %0.2f" % (scores.mean(), scores.std()))

# Gaussian Naive Bayes

gnb_model = GaussianNB()
gnb_model.fit(X, y)
scores = cross_val_score(gnb_model, X, y, cv=5)
print("Gaussian Naive Bayes: %0.2f accuracy with a standard deviation of %0.2f" % (scores.mean(), scores.std()))

# Support Vector Machines

svm_model = SVC(kernel='linear', gamma = 'auto')
svm_model.fit(X, y)
scores = cross_val_score(svm_model, X, y, cv=5)
print("Support Vector Machines: %0.2f accuracy with a standard deviation of %0.2f" % (scores.mean(), scores.std()))

# Random Forest Classifier

rfc_model = RandomForestClassifier()
rfc_model.fit(X, y)
scores = cross_val_score(rfc_model, X, y, cv=5)
print("Random Forest Classifier: %0.2f accuracy with a standard deviation of %0.2f" % (scores.mean(), scores.std()))

Logistic Regression: 0.35 accuracy with a standard deviation of 0.01
Linear Discriminant Analysis: 0.35 accuracy with a standard deviation of 0.02
K-Nearest Neighbors: 0.26 accuracy with a standard deviation of 0.03
Classification and Regression Trees: 0.23 accuracy with a standard deviation of 0.03
Gaussian Naive Bayes: 0.33 accuracy with a standard deviation of 0.03
Support Vector Machines: 0.34 accuracy with a standard deviation of 0.00
Random Forest Classifier: 0.28 accuracy with a standard deviation of 0.03
